<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/Next_Sentence_prediction_load_model_%EC%8B%9C%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 참고 --> https://www.kaggle.com/code/ysthehurricane/next-word-prediction-bi-lstm-tutorial-easy-way

import os
import numpy as np

# 하나의 문장 내에서 다음 단어들을 맞출 때 import할 것들
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Activation, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.models import load_model      # 저장한 모델 불러오기

# 단어가 아닌 sentence 단위로 전처리할 경우
import nltk
nltk.download('punkt')   # ‘punkt=문장 tokenizer’를 다운로드
from nltk.tokenize import word_tokenize, sent_tokenize
import re                # 파이썬 정규표현을 위한 표준 라이브러리임

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import files     # Colab에서 local PC내의 log file 읽어 들이기
uploaded = files.upload()          # 파일을 여러 개 선택 가능함

Saving test2.txt to test2.txt


In [ ]:
# log msg 를 sentence 단위로 토큰화 처리
input_sequences = []
log_sent = []

log_file_name='test2.txt'

'''--------------------------------------------------------------'''

f=open(log_file_name, 'r')
lines = f.readlines()

for line in lines:
    sent_tok =  nltk.sent_tokenize(line)     #  log msg를 줄 단위로 sentence로 토큰화
    log_sent.append(sent_tok)

print("log_sents=", log_sent)
print("num_sent=", len(log_sent))

tokenizer = Tokenizer(oov_token='<oov>')     #  토큰화할 때 적용할 파라미터 --> 없는 단어를 '<oov>'로 처리함.
tokenizer.fit_on_texts(log_sent)             #  전체 문장의 index와 단어 벡터를 생성(Keras)
total_words = len(tokenizer.word_index) + 1  #  계산된 단어(문장)의 index를 dictionary 형태(word_index)로 변환

print("total unique sentences=", total_words)
print("tokeniizer.word_index=", tokenizer.word_index)

for msg in log_sent:
    #print(msg)                                    # log msg를 한 줄씩 출력
    token_list = tokenizer.texts_to_sequences([msg])[0]  # 각 문장(line)에 index를 부여해줌.
    #print("token_list=",token_list)                              # --> [
    input_sequences.append(token_list)
print("input_sequences=", input_sequences)       # ---> [[92], [93], [94], [95], [96], [97], [40],....] 등으로 출력됨
print("Total input sequences: ", len(input_sequences))    # ---> log file의 line 수
f.close()


log_sents= [['starting rotate log files'], ['starting daily mandb regeneration'], ['mandbservice succeeded'], ['finished daily mandb regeneration'], ['logrotateservice succeeded'], ['finished rotate log files'], ['storehelpersgo cannot refresh snap has no updates available core lxd snapd'], ['autorefreshgo autorefresh all snaps are uptodate'], ['daemon system activating via systemd service nameorgfreedesktopfwupd unitfwupdservice requested by uid pid commusrbinfwupdmgr refresh nometadatacheck labelunconfined'], ['daemon system successfully activated service orgfreedesktopfwupd'], ['fetching metadata httpscdnfwupdorgdownloadsfirmwarexmlgz'], ['fetching signature httpscdnfwupdorgdownloadsfirmwarexmlgzasc'], ['successfully downloaded new metadata local devices supported'], ['starting refresh fwupd metadata and update motd'], ['starting firmware update daemon'], ['started firmware update daemon'], ['fwupdrefreshservice succeeded'], ['finished refresh fwupd metadata and update motd'], ['new

In [ ]:
# 모델 학습을 위한 Xs, ys 추출은 불필요함.

In [ ]:
from google.colab import files     # Colab에서 local PC내의 훈련 모델을  읽어 들임
uploaded = files.upload()         # 10MB  2분 소요
# 저장위치 :  /content/lstm_sent_predict.h5

In [ ]:
# https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ko
from tensorflow import keras
model = keras.models.load_model('/content/lstm_sent_predict.h5') # 훈련 모델을 다시 불러옴

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 200)            182800    
                                                                 
 bidirectional (Bidirectiona  (None, 300)              421200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 914)               275114    
                                                                 
Total params: 879,114
Trainable params: 879,114
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# matplotlib으로 학습과정을 그래프로 그리는 것은 학습과정(model.compile)에서 metric이 저장되지 않았으므로 사용 불가함.

# Model 시험하기

연속된 2개의 문장으로 한 개의 다음 문장 예측하기

In [ ]:
# 연속된 2개의 문장으로 한 개의 다음 문장 예측하기

seed_text = 'init ciinfo', 'init ciinfo route '

''' --------------------  실제 log msg를 파일에서 읽어 처리할 때  ------------------------'''
# model.pridict에 입력할 token_list 만들기
seed_tok=[]
for i in range(2):
    sd =  nltk.sent_tokenize(seed_text[i])   #  한 줄의 log msg를 sentence 단위로 토큰화
    seed_tok.append(sd)
print("seed_tok=", seed_tok)

token = tokenizer.texts_to_sequences(seed_tok)
print("token_list=", token)                 # 입력 문장의 index 값

token_list=[]
for a in token:
    token_list.append(a[0])
token_list=[token_list]
print("token_list=", token_list)             # token_list = [[1, 2]] 와 같은 2차원 배열로 표시되어야 함
'''-------------------------------------------------------------------------------------------'''

y_prob = model.predict(token_list, verbose=0)    # model.predicted()는 token_list(X-train)의 개수만큼 예측 결과를 출력함
print(y_prob[0][:20], len(y_prob[0]))                    # total_words(913개 unique 문장) 각각에 대한 확률 --> [[1.3384e-06 1.2275e-06 5.1810e-04 ... 3.392e-05]]


predicted = y_prob[0].argmax(axis=-1)            # 가장 큰 확률 값을 갖는 단어의 index를 선택
                                                    # axis가 0 이면 2차원 행렬에서 열(가장 높은 차원), axis=1이면 행(다음으로 높은 차원), axis=-1이면 마지막 차원(가장 낮은 차원, 2차원에서는 행)을 기준 축으로 함
                                                    # argmax()는 가장 큰 값이 아닌 그 값의 인덱스 값을 반환함.
print("predicted=", predicted)
prob= np.max(y_prob, axis=1)

#pred_rank = sorted(y_prob[0], reverse = True)                  #  값이 큰 것부터 정렬(reverse=True), sort()는 원본 자료를 수정하고, sorted()는 원본은 그대로 두고 수정한 것을 리턴함.
#print(pred_rank[0][:5])
#print(pred_rank)

output_word = ""
for word, index in tokenizer.word_index.items():   # 실제 단어로 변환하여 출력에 append 함.
    if index == predicted:
        output_word = word
        break
seed_text = output_word
print(seed_text, prob)


seed_tok= [['init ciinfo'], ['init ciinfo route']]
token_list= [[3], [1]]
token_list= [[3, 1]]
[1.0300854e-06 1.2203216e-06 4.4184503e-06 9.6786690e-01 7.3109049e-06
 1.8712765e-06 1.8790925e-05 5.3053518e-06 9.3162311e-07 4.8569264e-06
 1.8277681e-06 1.1951645e-05 2.0325801e-05 5.8447733e-07 1.2526841e-05
 3.5367996e-06 5.5209607e-06 3.6956033e-06 9.2586297e-06 8.4288916e-07] 914
predicted= 3
init ciinfo [0.9678669]


연속된 2개의 문장으로 다수의 Next Sentence를 예측하기

In [ ]:

seed_text = 'systemd stopped target default', 'systemd stopped target basic system'

num_to_predict = 5    # 예측할  다음 문장의 개수
''' --------------------  실제 log msg를 파일에서 읽어 처리할 때  ------------------------'''
# model.pridict에 입력할 token_list 만들기
seed_tok=[]
for i in range(2):
    sd =  nltk.sent_tokenize(seed_text[i])   #  한 줄의 log msg를 sentence 단위로 토큰화
    seed_tok.append(sd)
print("seed_tok=", seed_tok)

token = tokenizer.texts_to_sequences(seed_tok)
print("token_list=", token)                 # 입력 문장의 index 값

token_list=[]
for a in token:
    token_list.append(a[0])
token_list=[token_list]
print("token_list=", token_list)            # token_list = [[1, 2]] 와 같은 2차원 배열로 표시되어야 함
'''-------------------------------------------------------------------------------------------'''
print("")

y_prob = model.predict(token_list, verbose=0)    # model.predicted_classes 는 deprecated
#print(y_prob)

for i in range(num_to_predict):
    predicted = y_prob[0].argmax(axis=-1)
    print("predicted index =", predicted)

    prob= np.max(y_prob, axis=1)         # 2차원 array인 경우에 axis 사용함, 하나의 row에서 최대인 값
    np.put(y_prob[0], predicted, 0)         # np.array는 요소를 delete를 하면 새 array를 return하고 원본에서는 삭제가 안된다. 단, np.put(array, index, value) 등으로 변경은 가능하다.
    #np.delete(y_prob, predicted)

    output_word = ""
    for word, index in tokenizer.word_index.items():   # 실제 단어로 변환하여 출력에 append 함.
        if index == predicted:
            output_word = word
            break
    print("[", i+1,"번 ]", output_word, prob)


seed_tok= [['systemd stopped target default'], ['systemd stopped target basic system']]
token_list= [[1], [1]]
token_list= [[1, 1]]

predicted index = 143
[ 1 번 ] lvmlvmpolldsocket succeeded [0.01044711]
predicted index = 145
[ 2 번 ] systemdrfkillsocket succeeded [0.00890795]
predicted index = 343
[ 3 번 ] xmm memory block size mb [0.00722101]
predicted index = 841
[ 4 번 ] started getty on tty [0.00683403]
predicted index = 262
[ 5 번 ] ioapic apicid version address xfec gsi [0.00614791]


# EOF

(참고) Google Drive에 model 저장하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/lstm_sent_predict.h5') # Colab 좌측 창에 임시 저장된 h5 파일을 다운로드 하여 저장한다.

(참고) Source Code 저장하기

In [ ]:
# 인용--> https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221564411127&categoryNo=49&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

from os import path
from google.colab import drive

notebooks_dir_name = 'Colab Notebooks'
drive.mount('/content/drive')
notebooks_base_dir = path.join('./drive/My Drive/', notebooks_dir_name)
if not path.exists(notebooks_base_dir):
    print('Check your google drive directory"')

with open(path.join(notebooks_base_dir, "파일 이름"),"w") as f:
          f.write("Colab Keras code saved")

In [ ]:
from keras.models import load_model

model.save('lstm_sent_predict')  # 모델을 저장

del model       # 테스트를 위해 Colab 메모리 내의 모델을 삭제
loaded_model = model.load('lstm_sent_predict') # 모델을 새로 불러옴

print("\n Test Accuracy: %.4f" % (loaded_model.evaluate(X_test, Y_test)[1]))  # 불러온 모델로 테스트 실행